### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
# import libraries
import pandas as pd
import ads
from ads.dataset.factory import DatasetFactory

In [2]:
# set up a dictionary containing credentials required for the ADW connection
conn = {
        "user_name": "admin",
        "password": "Welcome#1234",
        "service_name": "adwck1_high", # the service levels can be found in <ADW wallet folder>/tnsnames.ora as one of the variable names to which the connection string value is assigned
        "wallet_location": "/home/datascience/Wallet_adwck.zip",
    }


In [3]:
list_of_tables =[ 'rap_table','allergy_features','careplan_features1','claim_features1','claim_features2','claim_features3','conditions_features','devices_features','immunization_features1','medications_features1','patients_tmp1','procedures_features1','encounters_6mo1','encounters_1yr1',]

In [4]:
# import dataset from Autonomous Data Warehouse (ADW)
for tab in list_of_tables:
    print("---table ...  ",tab)
    df = pd.DataFrame.ads.read_sql('select * from admin.'+tab, connection_parameters=conn)
    df.to_csv("../feature_csvs/"+tab+".csv",index=False)

---table ...   rap_table
---table ...   allergy_features
---table ...   careplan_features1
---table ...   claim_features1
---table ...   claim_features2
---table ...   claim_features3
---table ...   conditions_features
---table ...   devices_features
---table ...   immunization_features1
---table ...   medications_features1
---table ...   patients_tmp1
---table ...   procedures_features1
---table ...   encounters_6mo1
---table ...   encounters_1yr1


## Read rap table

In [5]:
sample_table = pd.read_csv("../feature_csvs/rap_table.csv")

In [6]:
sample_table.READMISSION.value_counts()

N    23007
Y     3589
Name: READMISSION, dtype: int64

## Read Allergy features

In [7]:
allergy = pd.read_csv("../feature_csvs/allergy_features.csv")

In [8]:
allergy.head()

,PATIENT,CODE_NEW,DESCRIPTION_NEW,DESCRIPTION1,DESCRIPTION2
0,4728942a-94dc-dcd6-3627-9fa8dc717464,412071004,Wheat (substance),NaN,NaN
1,64c23f1f-b5f2-14fe-b452-f0aaebb61b39,412071004,Wheat (substance),NaN,NaN
2,5e856998-8bd2-23e9-2674-b8c5f457c410,84489001,Mold (organism),Allergic skin rash,Sneezing
3,5e856998-8bd2-23e9-2674-b8c5f457c410,260147004,House dust mite (organism),NaN,NaN
4,5e856998-8bd2-23e9-2674-b8c5f457c410,264287008,Animal dander (substance),Rhinoconjunctivitis (disorder),NaN


In [9]:
tmp1=allergy[['PATIENT','CODE_NEW']].pivot_table(index=["PATIENT"], columns="CODE_NEW", aggfunc=lambda x: 1, fill_value=0).reset_index()

In [10]:
tmp1.columns

Index([      'PATIENT',            1191,            5640,            7984,
                 10831,           25037,           29046,         3718001,
              84489001,       102263004,       111088007,       256277009,
             256355007,       260147004,       264287008,       288328004,
             412071004,       735029006,       735971005,       762952008,
             782576004, 442571000124108],
      dtype='object', name='CODE_NEW')

In [11]:
tmp1.columns=['PATIENT']+['allergy_'+str(col) for col in tmp1.columns if col!='PATIENT']

In [12]:
sample_table.head()

,PATIENT,START_NEW,STOP,INP_DURATION,READMISSION
0,0360958b-500a-8b24-07dc-c6ec34186b7e,1914-10-28 00:56:04,1914-10-31 11:54:54,3.457523,N
1,56d5a077-96c5-a053-a55c-292484e5fd67,1920-01-12 14:49:07,1920-01-16 21:54:25,4.295347,N
2,11a9315b-b71a-d87e-56cf-4ffb471b2523,1920-12-17 21:23:30,1920-12-22 18:32:16,4.881088,N
3,223aa945-f14b-6006-7e7e-98ba1fb22cb6,1921-05-31 06:44:06,1921-06-04 07:43:45,4.041424,N
4,01318a0b-4cbb-3467-9605-dc8aeb7df935,1921-06-03 02:17:03,1921-06-04 22:58:48,1.862326,N


In [13]:
tmp2 = pd.merge(sample_table, tmp1, on =['PATIENT'], how='left')

## Read Care Plan features

In [14]:
careplan = pd.read_csv("../feature_csvs/careplan_features1.csv")

In [15]:
careplan.head()

,PATIENT,CODE_NEW,CAREPLAN_CNT
0,00002142-ea7b-b4a6-8427-69d5d9804e97,736353004,2
1,000475c5-6d1f-c2af-9324-f51afef719b7,737471002,1
2,00086d10-d97d-7062-1a93-27ae02b8e972,735984001,1
3,00086d10-d97d-7062-1a93-27ae02b8e972,736353004,1
4,0008934e-2eea-cba9-b210-55c3d794b272,134435003,1


In [16]:
import numpy as np
tmp3=careplan.pivot_table(index=["PATIENT"], columns="CODE_NEW",values='CAREPLAN_CNT',aggfunc=np.sum, fill_value=0).reset_index()

In [17]:
tmp3.columns=['PATIENT']+['careplan_'+str(col) for col in tmp3.columns if col!='PATIENT']

In [18]:
tmp4 = pd.merge(tmp2, tmp3, on =['PATIENT'], how='left')

## Read Claim_Features1 

In [19]:
diagnosis1 = pd.read_csv("../feature_csvs/claim_features1.csv")

In [20]:
diagnosis1.head()

,PATIENT,DIAGNOSIS1,DIAGNOSIS1_CNT
0,00002142-ea7b-b4a6-8427-69d5d9804e97,160903007,1
1,000475c5-6d1f-c2af-9324-f51afef719b7,698314001,1
2,00071b57-6e95-fcc4-8450-e8f1c78bc297,224299000,1
3,00086d10-d97d-7062-1a93-27ae02b8e972,88805009,3
4,000edd71-0f16-a819-8c74-c1a9ef7e5773,10509002,1


In [21]:
tmp6=diagnosis1.pivot_table(index=["PATIENT"], columns="DIAGNOSIS1",values='DIAGNOSIS1_CNT',aggfunc=np.sum, fill_value=0).reset_index()
tmp6.columns=['PATIENT']+['diagnosis1_'+str(col) for col in tmp6.columns if col!='PATIENT']

In [22]:
diagnosis2 = pd.read_csv("../feature_csvs/claim_features2.csv")
tmp7=diagnosis2.pivot_table(index=["PATIENT"], columns="DIAGNOSIS2",values='DIAGNOSIS2_CNT',aggfunc=np.sum, fill_value=0).reset_index()
tmp7.columns=['PATIENT']+['diagnosis2_'+str(col) for col in tmp7.columns if col!='PATIENT']

diagnosis3 = pd.read_csv("../feature_csvs/claim_features3.csv")
tmp8=diagnosis3.pivot_table(index=["PATIENT"], columns="DIAGNOSIS3",values='DIAGNOSIS3_CNT',aggfunc=np.sum, fill_value=0).reset_index()
tmp8.columns=['PATIENT']+['diagnosis3_'+str(col) for col in tmp8.columns if col!='PATIENT']

In [23]:
tmp6.shape, tmp7.shape, tmp8.shape

((26596, 195), (22786, 144), (7125, 81))

In [24]:
from functools import reduce
tmp9 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp4, tmp6, tmp7, tmp8])

## Read Conditions Features

In [25]:
conditions = pd.read_csv("../feature_csvs/conditions_features.csv")

In [26]:
conditions.head()

,PATIENT,CODE1,DESCRIPTION1
0,22222390-353f-b885-8c5a-61b0ce74f2cd,162864005,Body mass index 30+ - obesity (finding)
1,22222390-353f-b885-8c5a-61b0ce74f2cd,424132000,Non-small cell carcinoma of lung TNM stage 1 (disorder)
2,5004fa7a-d4d7-b295-7ae7-989010de09c2,15777000,Prediabetes
3,1fd75137-b477-583d-0a0d-875d28674625,423315002,Limited social contact (finding)
4,b4f41dbf-b8d3-f3c0-1cbd-799523bea1d7,230690007,Stroke


In [27]:
tmp10=conditions[['PATIENT','CODE1']].pivot_table(index=["PATIENT"], columns="CODE1", aggfunc=lambda x: 1, fill_value=0).reset_index()
tmp10.columns=['PATIENT']+['Condition_'+str(col) for col in tmp10.columns if col!='PATIENT']
tmp11 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp9, tmp10])

## Devices Features

In [28]:
Devices = pd.read_csv("../feature_csvs/devices_features.csv")

In [29]:
Devices.head()

,PATIENT,CODE,DESCRIPTION
0,00002142-ea7b-b4a6-8427-69d5d9804e97,337414009,Blood glucose meter (physical object)
1,00086d10-d97d-7062-1a93-27ae02b8e972,170615005,Home nebulizer (physical object)
2,000edd71-0f16-a819-8c74-c1a9ef7e5773,170615005,Home nebulizer (physical object)
3,000edd71-0f16-a819-8c74-c1a9ef7e5773,337414009,Blood glucose meter (physical object)
4,00165064-8b0e-546b-398d-5c9ec64aee13,706004007,Implantable cardiac pacemaker (physical object)


In [35]:
# tmp12=Devices[['PATIENT','CODE_NEW']].pivot_table(index=["PATIENT"], columns="CODE_NEW", aggfunc=lambda x: 1, fill_value=0).reset_index()
tmp12=Devices[['PATIENT','CODE']].pivot_table(index=["PATIENT"], columns="CODE", aggfunc=len, fill_value=0).reset_index()


In [36]:
tmp12.columns=['PATIENT']+['Devices_'+str(col) for col in tmp12.columns if col!='PATIENT']
tmp13 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp11, tmp12])

# Encounters Features

In [37]:
Encounter_1yr = pd.read_csv("../feature_csvs/encounters_1yr1.csv")

In [38]:
Encounter_1yr.head()

,PATIENT,CODE,ENCOUNTER_CNT
0,000475c5-6d1f-c2af-9324-f51afef719b7,698314001,1
1,00071b57-6e95-fcc4-8450-e8f1c78bc297,50849002,1
2,00071b57-6e95-fcc4-8450-e8f1c78bc297,162673000,1
3,00086d10-d97d-7062-1a93-27ae02b8e972,185347001,3
4,000edd71-0f16-a819-8c74-c1a9ef7e5773,50849002,1


In [39]:
tmp14=Encounter_1yr.pivot_table(index=["PATIENT"], columns="CODE", values='ENCOUNTER_CNT', aggfunc=np.sum, fill_value=0).reset_index()
tmp14.columns=['PATIENT']+['Encounter_1yr_'+str(col) for col in tmp14.columns if col!='PATIENT']
# tmp15 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp13, tmp14])

In [40]:
Encounter_6mo = pd.read_csv("../feature_csvs/encounters_6mo1.csv")

In [41]:
tmp15=Encounter_6mo.pivot_table(index=["PATIENT"], columns="CODE", values='ENCOUNTER_CNT', aggfunc=np.sum, fill_value=0).reset_index()
tmp15.columns=['PATIENT']+['Encounter_6mo_'+str(col) for col in tmp15.columns if col!='PATIENT']
tmp16 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp13, tmp14,tmp15])

In [42]:
tmp16.shape

(26596, 819)

## Reading Immunizations

In [44]:
Immunizations = pd.read_csv("../feature_csvs/immunization_features1.csv")

In [45]:
tmp17=Immunizations.pivot_table(index=["PATIENT"], columns="CODE", values ='IMMUNIZATIONS_CNT', aggfunc=np.sum, fill_value=0).reset_index()
tmp17.columns=['PATIENT']+['Immunizations_'+str(col) for col in tmp17.columns if col!='PATIENT']
tmp18 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp16, tmp17])


## Reading Medications

In [46]:
Medications = pd.read_csv("../feature_csvs/medications_features1.csv")

In [47]:
Medications.head()

,PATIENT,CODE,DISPENSES_CNT
0,00086d10-d97d-7062-1a93-27ae02b8e972,313988,23.315788
1,000edd71-0f16-a819-8c74-c1a9ef7e5773,2001499,19.810588
2,001282ec-a1aa-d754-eb96-80f64cea4771,310798,23.676550
3,001282ec-a1aa-d754-eb96-80f64cea4771,314076,732.000000
4,001282ec-a1aa-d754-eb96-80f64cea4771,849574,11.934783


In [48]:
tmp19=Medications.pivot_table(index=["PATIENT"], columns="CODE", values ='DISPENSES_CNT', aggfunc=np.sum, fill_value=0).reset_index()
tmp19.columns=['PATIENT']+['Medications_'+str(col) for col in tmp19.columns if col!='PATIENT']
tmp20 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp18, tmp19])


## Reading PATIENTS

In [49]:
Patients = pd.read_csv("../feature_csvs/patients_tmp1.csv")

In [50]:
Patients.head()

,PATIENT,START_NEW,BIRTHDATE,AGE,AGE1,RACE,ETHNICITY,GENDER
0,73663db7-49ea-78fb-f258-229b65b458ed,1967-01-11 16:04:48,1966-12-21 00:00:00,0,21.67,white,nonhispanic,F
1,8946bdc9-44fe-89a3-58a3-128a501527aa,2020-11-19 22:04:48,2017-12-05 00:00:00,2,1080.92,white,nonhispanic,F
2,38a4c271-2848-7c9b-68ec-516f9624f728,1981-01-19 23:31:12,1966-12-07 00:00:00,14,5157.98,white,hispanic,F
3,1587c79f-e68e-cde2-181f-53c2d41d134a,1999-04-20 20:52:48,1985-01-16 00:00:00,14,5207.87,white,nonhispanic,M
4,f54ecf07-daab-eb39-02a5-b8367f8db8d9,1999-09-17 20:52:48,1985-01-16 00:00:00,14,5357.87,white,nonhispanic,M


In [51]:
tmp21 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp20, Patients[['PATIENT','AGE','RACE','ETHNICITY','GENDER']]])

## Reading Procedures

In [52]:
Procedures = pd.read_csv("../feature_csvs/procedures_features1.csv")

In [53]:
Procedures.head()

,PATIENT,CODE,PROCEDURES_CNT
0,00002142-ea7b-b4a6-8427-69d5d9804e97,454711000124102,1
1,000475c5-6d1f-c2af-9324-f51afef719b7,252160004,1
2,00086d10-d97d-7062-1a93-27ae02b8e972,29303009,1
3,00086d10-d97d-7062-1a93-27ae02b8e972,73761001,1
4,00086d10-d97d-7062-1a93-27ae02b8e972,763302001,1


In [54]:
tmp22=Procedures.pivot_table(index=["PATIENT"], columns="CODE", values ='PROCEDURES_CNT', aggfunc=np.sum, fill_value=0).reset_index()
tmp22.columns=['PATIENT']+['Procedures_'+str(col) for col in tmp22.columns if col!='PATIENT']
tmp23 = reduce(lambda x,y: pd.merge(x,y, on='PATIENT', how='left'), [tmp21, tmp22])


In [55]:
tmp23.shape

(26596, 1210)

In [56]:
tmp23.head()

,PATIENT,START_NEW,STOP,INP_DURATION,READMISSION,allergy_1191,allergy_5640,allergy_7984,allergy_10831,allergy_25037,...,Procedures_763302001,Procedures_866148006,Procedures_868187001,Procedures_112001000119100,Procedures_112011000119102,Procedures_426701000119108,Procedures_428211000124100,Procedures_449381000124108,Procedures_454711000124102,Procedures_16335031000119103
0,0360958b-500a-8b24-07dc-c6ec34186b7e,1914-10-28 00:56:04,1914-10-31 11:54:54,3.457523,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56d5a077-96c5-a053-a55c-292484e5fd67,1920-01-12 14:49:07,1920-01-16 21:54:25,4.295347,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11a9315b-b71a-d87e-56cf-4ffb471b2523,1920-12-17 21:23:30,1920-12-22 18:32:16,4.881088,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,223aa945-f14b-6006-7e7e-98ba1fb22cb6,1921-05-31 06:44:06,1921-06-04 07:43:45,4.041424,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01318a0b-4cbb-3467-9605-dc8aeb7df935,1921-06-03 02:17:03,1921-06-04 22:58:48,1.862326,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
ignore_cols =['START_NEW','STOP','INP_DURATION']

In [58]:
tmp24=tmp23.drop(columns=ignore_cols)

In [59]:
tmp24.shape

(26596, 1207)

In [60]:
len(np.unique(tmp24.PATIENT.values))

26596

In [61]:
tmp24.fillna(0,inplace=True)

In [62]:
tmp24.to_csv("../feature_csvs/RAP_DF.csv",index=None)

In [63]:
tmp24.head()

,PATIENT,READMISSION,allergy_1191,allergy_5640,allergy_7984,allergy_10831,allergy_25037,allergy_29046,allergy_3718001,allergy_84489001,...,Procedures_763302001,Procedures_866148006,Procedures_868187001,Procedures_112001000119100,Procedures_112011000119102,Procedures_426701000119108,Procedures_428211000124100,Procedures_449381000124108,Procedures_454711000124102,Procedures_16335031000119103
0,0360958b-500a-8b24-07dc-c6ec34186b7e,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,56d5a077-96c5-a053-a55c-292484e5fd67,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11a9315b-b71a-d87e-56cf-4ffb471b2523,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,223aa945-f14b-6006-7e7e-98ba1fb22cb6,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01318a0b-4cbb-3467-9605-dc8aeb7df935,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Above would create ML ready data frame. 



# Next use 001_build_model.ipynb in model_build folder for building, cataloging and deploying the model

In [64]:
tmp24 = pd.read_csv("../feature_csvs/RAP_DF.csv")

In [65]:
tmp24.columns

Index(['PATIENT', 'READMISSION', 'allergy_1191', 'allergy_5640',
       'allergy_7984', 'allergy_10831', 'allergy_25037', 'allergy_29046',
       'allergy_3718001', 'allergy_84489001',
       ...
       'Procedures_763302001', 'Procedures_866148006', 'Procedures_868187001',
       'Procedures_112001000119100', 'Procedures_112011000119102',
       'Procedures_426701000119108', 'Procedures_428211000124100',
       'Procedures_449381000124108', 'Procedures_454711000124102',
       'Procedures_16335031000119103'],
      dtype='object', length=1207)

In [66]:
len([col for col in tmp24.columns if col.startswith('allergy')]), len([col for col in tmp24.columns if col.startswith('care')]),

(21, 37)

In [67]:
len([col for col in tmp24.columns if col.startswith('diagnosis1')]),len([col for col in tmp24.columns if col.startswith('diagnosis2')]), 

(194, 143)

In [68]:
len([col for col in tmp24.columns if col.startswith('diagnosis3')]),len([col for col in tmp24.columns if col.startswith('patient')]), 

(80, 0)

In [69]:
len([col for col in tmp24.columns if col.startswith('Condition')]),len([col for col in tmp24.columns if col.startswith('Devices')]), 

(230, 7)

In [70]:
len([col for col in tmp24.columns if col.startswith('Encounter')]),len([col for col in tmp24.columns if col.startswith('Immunizations')]), 

(102, 21)

In [71]:
len([col for col in tmp24.columns if col.startswith('Medications')]),len([col for col in tmp24.columns if col.startswith('Procedures')]), 

(173, 193)

# Useful code

In [7]:
# convert from Pandas Dataframe to ADSDataset
ds = DatasetFactory.from_dataframe(df)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# data exploration
ds.show_in_notebook()

Accordion(children=(HTML(value='<h1>Name: User Provided DataFrame</h1><h3>Type: ADSDataset</h3><h3>26,596 Rows…

In [ ]:
# convert back to Pandas Dataframe
df1 = ds.to_pandas_dataframe()

In [ ]:
# export dataset to ADW
df1.ads.to_sql('<ADW username of table owner>.<name of table to post>', connection_parameters=conn, if_exists='replace')